In [1]:
import os
os.getcwd()

'/home/zhalgas.zhienbekov'

In [2]:
os.chdir('shared_risk_folder/ZZ_Projects/Connection_score/')

In [3]:
import time
import datetime
import subprocess
from datetime import datetime
import pickle
import numpy as np
#from pyspark_job_control.pyspark_job_control_logs import JobControlLogs
from pyspark.sql.functions import col
from core_functions import (
    get_config,
    set_environment_variables,
    update_impala_metadata,
    create_spark_session,
    dwh_update,
    dwh_insert,
    create_html_table,
    send_email )

In [4]:
PROJECT_PATH = os.getcwd()

In [5]:
src_db_nm="bdp_feature_offline_stg"
src_table_nm="report_month_t_zz_connect_score_features_1"

trg_sys_name="bdp_score_stg"
trg_tab_name="daily_connection_score23_hdfs"
dwh_table_name="OWNER_EXT.T_ZZ_CONNECTION_SCORE_ACTUAL"
pickle_file_path_1 = f"{PROJECT_PATH}/model_connect_score_24_1_final.pkl"
pickle_file_path_2 = f"{PROJECT_PATH}/model_connect_score_24_2_final.pkl"

In [6]:
SMTP_SERVER = 'smtp-int.itc.homecredit.kz'
SMTP_PORT = 25

In [7]:
appName="init_connection_score_generator"
spark = create_spark_session(appName)

In [8]:
df = spark.sql("select * from bdp_feature_offline_stg.report_month_t_zz_connect_score_features_1")

In [9]:
pd_data = df.toPandas()

In [10]:
pd_data.head()

,id_cuid,phone_number,pif_phone_type_code,rate_out_calls_6m,cnt_calls_1m,past_days_last_succ_call,cnt_calls_18m,cnt_success_sms_6m,avg_talk_18m,last_interaction_type_encoded,unsuccess_class,part_date
0,3180354,7010228181,PRIMARY_MOBILE,0.0,0,-1,0,0,0E-10,2,0,20240804
1,2723760,7027586061,PRIMARY_MOBILE,0.0,0,-1,0,0,0E-10,2,0,20240804
2,1013991,7052711822,PRIMARY_MOBILE,0.0,0,-1,0,0,0E-10,2,0,20240804
3,2205338,7056716882,PRIMARY_MOBILE,0.0,0,-1,0,0,0E-10,2,0,20240804
4,617623,70766480886,PRIMARY_MOBILE,0.0,0,-1,0,0,0E-10,2,0,20240804


In [11]:
pd_data['avg_talk_18m'] = pd_data['avg_talk_18m'].astype(float)

In [13]:
pd_data['avg_talk_18m'].mean()

7.009612860186319

In [14]:
with open(pickle_file_path_1, "rb") as file1:
    model_cs_1 = pickle.load(file1)
with open(pickle_file_path_2, "rb") as file2:
    model_cs_2 = pickle.load(file2)

In [15]:
predictors1 = model_cs_1.feature_name()
predictors2 = model_cs_2.feature_name()

In [16]:
pd_data[['phone_number']] = pd_data[['phone_number']].applymap(lambda x: str(x).strip().replace(' ', ''))
pd_data['phone_number'] = pd_data['phone_number'].astype(str)
pd_data = pd_data[(pd_data['phone_number'].str.len() == 11) | (pd_data['phone_number'].str.len() == 10)]

In [17]:
pd_data1 = pd_data[pd_data['pif_phone_type_code'] == 'PRIMARY_MOBILE'][predictors1]
pd_data2 = pd_data[predictors2]

In [18]:
pd_data1.dtypes

rate_out_calls_6m                float64
cnt_calls_1m                       int64
past_days_last_succ_call           int32
cnt_calls_18m                      int64
cnt_success_sms_6m                 int64
avg_talk_18m                     float64
last_interaction_type_encoded       int8
unsuccess_class                    int32
dtype: object

In [19]:
pd_data1[['unsuccess_class', 'last_interaction_type_encoded']] = pd_data1[['unsuccess_class', 'last_interaction_type_encoded']].astype('category')
pd_data2[['unsuccess_class', 'last_interaction_type_encoded']] = pd_data2[['unsuccess_class', 'last_interaction_type_encoded']].astype('category')

/opt/cloudera/parcels/ANACONDA/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [20]:
pd_data1['rate_out_calls_6m'] = pd_data1['rate_out_calls_6m'].fillna(0)
pd_data2['rate_out_calls_6m'] = pd_data2['rate_out_calls_6m'].fillna(0)

/opt/cloudera/parcels/ANACONDA/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
print(pd_data1.shape)
print(pd_data2.shape)

(9638228, 8)
(14845457, 7)


In [96]:
data_scores1 = pd_data[pd_data['pif_phone_type_code'] == 'PRIMARY_MOBILE'].reset_index()[['phone_number','id_cuid']]
data_scores2 = pd_data[['phone_number','id_cuid']]
print(data_scores1.shape)
print(data_scores2.shape)

(9638228, 2)
(14845457, 2)


In [117]:
models_list = [model_cs_1, model_cs_2]
pd_list = [pd_data1, pd_data2]
data_list = [data_scores1, data_scores2]
dict_scores = {}

In [122]:
for i in range(2):
    score_primary = models_list[i].predict(pd_list[i])
    data_list[i]['SCORE'] = score_primary
    data_list[i]['SK_DTIME_SCORE'] = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
    data_list[i]['SK_DATE_SCORE'] = datetime.now().strftime("%Y%m%d")
    data_list[i]['DATE_'] = datetime.now().strftime("%d.%m.%Y")
    data_list[i] = data_list[i].rename(columns = lambda x:x.upper())
    data_list[i]['PHONE_NUMBER'] = data_list[i]['PHONE_NUMBER'].astype(str)
    data_list[i]['ID_CUID'] = data_list[i]['ID_CUID'].astype(int)
    
    dict_scores[i] = data_list[i].sort_values(by='SCORE', ascending=False).reset_index(drop=True)
    num_bins = 10
    bin_edges = np.linspace(0, len(dict_scores[i]), num_bins + 1).astype(int)
    dict_scores[i]['SCORE_GROUP'] = 0
    for j in range(num_bins):
        dict_scores[i].loc[bin_edges[j]:bin_edges[j+1]-1, 'SCORE_GROUP'] = j + 1
    dict_scores[i]['SK_DATE_SCORE'] = dict_scores[i]['SK_DATE_SCORE'].astype(int)
    dict_scores[i] = dict_scores[i][['PHONE_NUMBER','ID_CUID','DATE_','SCORE','SK_DATE_SCORE','SCORE_GROUP','SK_DTIME_SCORE']]
    print(i)

0
1


In [124]:
dict_scores[1].shape

(14845457, 7)

In [ ]:
score_primary = model_cs_1.predict(pd_data1)

In [35]:
data_scores1['SCORE'] = score_primary
data_scores1['SK_DTIME_SCORE'] = datetime.now().strftime("%d.%m.%Y %H:%M:%S")
data_scores1['SK_DATE_SCORE'] = datetime.now().strftime("%Y%m%d")
data_scores1['DATE_'] = datetime.now().strftime("%d.%m.%Y")
data_scores1 = data_scores1.rename(columns = lambda x:x.upper())
data_scores1['PHONE_NUMBER'] = data_scores1['PHONE_NUMBER'].astype(str)
data_scores1['ID_CUID'] = data_scores1['ID_CUID'].astype(int)

In [39]:
data_test = data_scores1.sort_values(by='SCORE', ascending=False).reset_index(drop=True)
num_bins = 10
bin_edges = np.linspace(0, len(data_test), num_bins + 1).astype(int)
data_test['SCORE_GROUP'] = 0
for i in range(num_bins):
    data_test.loc[bin_edges[i]:bin_edges[i+1]-1, 'SCORE_GROUP'] = i + 1

In [41]:
data_test['SK_DATE_SCORE'] = data_test['SK_DATE_SCORE'].astype(int)

In [43]:
data_test = data_test[['PHONE_NUMBER','ID_CUID','DATE_','SCORE','SK_DATE_SCORE','SCORE_GROUP','SK_DTIME_SCORE']]

In [61]:
data_test.head()

,PHONE_NUMBER,ID_CUID,DATE_,SCORE,SK_DATE_SCORE,SCORE_GROUP,SK_DTIME_SCORE
0,7781710895,8172678,06.08.2024,0.775687,20240806,1,06.08.2024 13:59:12
1,7755721972,4119058,06.08.2024,0.771852,20240806,1,06.08.2024 13:59:12
2,7073180655,4953636,06.08.2024,0.769304,20240806,1,06.08.2024 13:59:12
3,7788472835,9458346,06.08.2024,0.765745,20240806,1,06.08.2024 13:59:12
4,7076243805,1547585,06.08.2024,0.765298,20240806,1,06.08.2024 13:59:12


In [62]:
data_test.shape

(9638228, 7)

In [58]:

def dwh_connection_AP_RISK(login, password):
    """
    Inner-function for functions dwh_update and dwh_insert
    f'{login}/{password}@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=DBHDWKZ.KZ.PROD)(PORT=1521))(CONNECT_DATA=(UR=A)(SERVER=DEDICATED)(SERVICE_NAME=HDWKZ.KZ.PROD)))',
    """
    connection = cx_Oracle.connect(
        f'{login}/{password}@DBHDWKZ.KZ.PROD:1521/HDWKZ.KZ.PROD',
        encoding='UTF-8',
        nencoding='UTF-8'
    )
    return connection


def dwh_update(login, password):
    """
    Function to truncate a table via procedure
    """
    conn_ap_risk = dwh_connection_AP_RISK(login, password)
    cursor = cx_Oracle.Cursor(conn_ap_risk)

    sql_truncate = '''begin
    OWNER_EXT.TRUNCATE_TABLE_CONNECTION_SCORE_ACTUAL;
    end;'''
    cursor.execute(sql_truncate)

    conn_ap_risk.commit()

    cursor.close()
    conn_ap_risk.close()

def dwh_insert(data, login, password):
    """
    Function to insert data into table
    """
    conn_ap_risk = dwh_connection_AP_RISK(login, password)
    cursor = cx_Oracle.Cursor(conn_ap_risk)
    df = data.astype(str)
    rowss = df[['PHONE_NUMBER', 'ID_CUID', 'DATE_', 'SCORE', 'SK_DATE_SCORE', 'SCORE_GROUP', 'SK_DTIME_SCORE']].to_records(index=False).tolist()

    # cursor.prepare('''insert into ap_risk.t_zz_connection_score_history (PHONE_NUMBER, ID_CUID, DATE_, SCORE, SK_DATE_SCORE, SCORE_GROUP, SK_DTIME_SCORE) values (:1, :2, to_date(:3, 'dd.mm.yyyy'), :4, :5, :6, to_date(:7, 'dd.mm.yyyy HH24:MI:SS') )''')
    # cursor.executemany(None, rowss)

    cursor.prepare('''insert into OWNER_EXT.T_ZZ_CONNECTION_SCORE_P_ACTUAL (PHONE_NUMBER, ID_CUID, DATE_, SCORE, SK_DATE_SCORE, SCORE_GROUP, SK_DTIME_SCORE) values (:1, :2, to_date(:3, 'dd.mm.yyyy'), :4, :5, :6, to_date(:7, 'dd.mm.yyyy HH24:MI:SS') )''')
    cursor.executemany(None, rowss)

    conn_ap_risk.commit()
    cursor.close()
    conn_ap_risk.close()


In [ ]:
#dwh_update(login=dm_bdp_etl_user, password=dm_bdp_etl_pass)

In [59]:
import getpass
import cx_Oracle

In [60]:
dm_bdp_etl_user = 'HCKZ_ZHALGASZHIENBEKOV[AP_CRM_ABT]'
dm_bdp_etl_pass =  getpass.getpass('Введите пароль: ')

Введите пароль:  ·················


In [133]:
int(dict_scores[0].shape[0]/2000000) + 1

5

In [62]:
#dwh_update(login=dm_bdp_etl_user, password=dm_bdp_etl_pass)
start_time = time.time()

for i in range(11):
    data_part = data_test.iloc[i*2000000:(i+1)*2000000]
    dwh_insert(data=data_part, login=dm_bdp_etl_user, password=dm_bdp_etl_pass)
    print(f'Inserted data from rows {i*2100000} to {(i+1)*2100000}')
    print('Loading...')
print('Success, table is ready to read!')

Inserted data from rows 0 to 2100000
Loading...
Inserted data from rows 2100000 to 4200000
Loading...
Inserted data from rows 4200000 to 6300000
Loading...
Inserted data from rows 6300000 to 8400000
Loading...
Inserted data from rows 8400000 to 10500000
Loading...
Inserted data from rows 10500000 to 12600000
Loading...
Inserted data from rows 12600000 to 14700000
Loading...
Inserted data from rows 14700000 to 16800000
Loading...
Inserted data from rows 16800000 to 18900000
Loading...
Inserted data from rows 18900000 to 21000000
Loading...
Inserted data from rows 21000000 to 23100000
Loading...
Success, table is ready to read!
